# Laboratoire 1: Introduction à PyTorch

Le but de se laboratoire est de se familiariser avec PyTorch en l'utilisant pour faire du classement sur deux jeux de données connus: MNIST et CIFAR-10.

In [ ]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import time
import matplotlib.pyplot as plt
%matplotlib inline

from deeplib.datasets import load_mnist, load_cifar10, train_valid_loaders
from sklearn.metrics import accuracy_score
from deeplib.net import MnistNet, CifarNet
from deeplib.history import History
from deeplib.visualization import plot_images

from torch.autograd import Variable
from torchvision.transforms import ToTensor
from torch.utils.data.sampler import SequentialSampler

## Mnist

Mnist est jeu de données contenant des images de chiffres manuscrits.<br>
Le jeu de données est séparé comme suit: 50000 images sont utilisées en entraînement et 10000 en test.

In [ ]:
mnist, mnist_test = load_mnist()

### Visualisation du jeu de données

Exécuter cette cellule plusieurs fois pour visualiser les données.

In [ ]:
idx = random.sample([x for x in range(len(mnist))], 9)
images = [np.array(mnist[i][0]) for i in idx]
targets = [mnist[i][1] for i in idx]

plot_images(images, targets, gray=True)

## Entraînement

Pour l'entraînement, nous avons besoin d'une fonction ``train`` pour entraîner le réseau et d'une fonction ``validate`` pour estimer la performance de notre modèle.

Pour chaque epoch, la fonction ``train`` passe au travers de toutes les images du jeu de données dans un ordre aléatoire et met à jour les poids du réseau selon la perte calculée.<br>
Pour entraîner le réseau, la fonction doit recevoir 3 hyperparamètres: 
1. le nombre d'epochs qui indique combien de fois toutes les images du jeu de données seront visualisées 
2. la taille de la batch qui indique combien d'images seront traitées à la fois
3. le taux d'apprentissage qui détermine la vitesse à laquelle chaque poids du réseau sera modifié

Pendant l'entraînement, une partie des données est utilisée pour créer un ensemble de validation qui permet d'estimer les performances de généralisation du modèle.

Finalement, on sauvegarde aussi quelques informations importantes afin de visualiser ce qui se passe pendant l'entraînement.

In [ ]:
def train(model, dataset, n_epoch, batch_size, learning_rate, use_gpu=False):
    history = History()
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    
    dataset.transform = ToTensor()
    train_loader, val_loader = train_valid_loaders(dataset, batch_size=batch_size)

    for i in range(n_epoch):
        model.train()
        for j, batch in enumerate(train_loader):
            
            inputs, targets = batch
            if use_gpu:
                inputs = inputs.cuda()
                targets = targets.cuda()

            optimizer.zero_grad()
            output = model(inputs)

            loss = criterion(output, targets)
            loss.backward()
            optimizer.step()
        
        train_acc, train_loss = validate(model, train_loader, use_gpu)
        val_acc, val_loss = validate(model, val_loader, use_gpu)
        history.save(train_acc, val_acc, train_loss, val_loss, learning_rate)
        print('Epoch {} - Train acc: {:.2f} - Val acc: {:.2f} - Train loss: {:.4f} - Val loss: {:.4f}'.format(i, train_acc, val_acc, train_loss, val_loss))
        
    return history



Pour chaque image du jeu de donnée, la fonction ``validate`` fait prédire une classe au réseau entraîné et compare le résultat avec la vraie réponse. Elle retourne le pourcentage de réponse correcte.

In [ ]:
def validate(model, val_loader, use_gpu=False):
    true =[]
    pred = []
    val_loss = []
    
    criterion = nn.CrossEntropyLoss()
    model.eval()
    
    for j, batch in enumerate(val_loader):
        
        inputs, targets = batch
        if use_gpu:
            inputs = inputs.cuda()
            targets = targets.cuda()
            
        output = model(inputs)
        
        predictions = output.max(dim=1)[1]
        
        val_loss.append(criterion(output, targets).data[0])
        true.extend(targets.data.cpu().numpy().tolist())
        pred.extend(predictions.data.cpu().numpy().tolist())

    return accuracy_score(true, pred) * 100, sum(val_loss) / len(val_loss)

Entraînons un modèle.

In [ ]:
model = MnistNet()

n_epoch = 10
batch_size = 64
learning_rate = 0.1

history = train(model, mnist, n_epoch, batch_size, learning_rate)

La fonction suivante permet de visualiser l'entraînement précédent.

Le premier graphique montre l'évolution de la précision du modèle sur le jeu de données d'entraînement et sur celui de validation. Le deuxième montre la perte sur les deux jeux de données.

In [ ]:
history.display()

Finalement, évaluons les performances du modèle sur le jeu de données de test.

In [ ]:
mnist_test.transform = ToTensor()
sampler = SequentialSampler(mnist_test)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, sampler=sampler)

score, loss = validate(model, test_loader)
print(score)

## Différences CPU - GPU

Pour faire exécuter le code sur GPU, il faut déplacer le model, les inputs et les targets sur le GPU. 

Le réseau contient deux couches de convolutions qui servent à extraire des caractéristiques des images tandis que les couches linéaires servent de classifieur. Il s'agit d'un pipeline commun pour toutes les classes.

In [ ]:
model_gpu = MnistNet()
model_gpu.cuda()

Comparons le temps d'exécution sur CPU et sur GPU

In [ ]:
epoch = 5
batch_size = 256
lr = 0.1

Pendant l'entraînement, vérifier l'utilisation du CPU avec la commande ``top``.

In [ ]:
print('Training on CPU')
model = MnistNet()

start_cpu = time.time()
history = train(model, mnist, epoch, batch_size, lr)
end_cpu = time.time()

cpu_time = end_cpu - start_cpu

Pour vous assurer que le réseau entraîne bien sur GPU, utiliser la commande 

``watch -n 1 nvidia-smi`` 

Observer l'utilisation de la carte et la quantité de mémoire utilisée pendant l'entraînement.

In [ ]:
print('Training on GPU')
model_gpu = MnistNet()
model_gpu.cuda()

start_gpu = time.time()
history_gpu = train(model_gpu, mnist, epoch, batch_size, lr, use_gpu=True)
end_gpu = time.time()

gpu_time = end_gpu - start_gpu

In [ ]:
print('CPU - Training time: {:.2f}s'.format(cpu_time))
print('GPU - Training time: {:.2f}s'.format(gpu_time))
print('Ratio: {:.2f}x'.format((cpu_time) / (gpu_time)))

## CIFAR 10

CIFAR-10 est un jeu de données comportant des images séparés en 10 classe:<br>
0 - Avion<br>
1 - Voiture<br>
2 - Oiseau<br>
3 - Chat<br>
4 - Chevreuil<br>
5 - Chien<br>
6 - Grenouille<br>
7 - Cheval<br>
8 - Bateau<br>
9 - Camion<br>

Le jeu de données contient 50000 images d'entraînement. Nous en utiliserons 40000 pour l'entraînement et 10000 pour la validation.

### Visualisation du jeu de données

Encore une fois, vous pouvez exécuter cette cellule plusieurs fois pour bien visualiser le jeu de données.

In [ ]:
cifar, cifar_test = load_cifar10()

In [ ]:
label_names = [
    'airplane',
    'automobile',
    'bird',
    'cat',
    'deer',
    'dog',
    'frog',
    'horse',
    'ship',
    'truck'
]

idx = random.sample([x for x in range(len(cifar))], 9)
images = [np.array(cifar[i][0]) for i in idx]
images = np.asarray(images)
targets = [cifar[i][1] for i in idx]

plot_images(images, targets, label_names)

### Exercices

Utilisez les 3 cellules suivantes pour répondre aux questions.

In [ ]:
epoch = 5
batch_size = 64
learning_rate = 0.1

model= CifarNet()
model.cuda()

history = train(model, cifar, epoch, batch_size, learning_rate, use_gpu=True)

In [ ]:
history.display()

In [ ]:
cifar_test.transform = ToTensor()
sampler = SequentialSampler(cifar_test)
test_loader = torch.utils.data.DataLoader(cifar_test, batch_size=batch_size, sampler=sampler)

score, loss = validate(model, test_loader, use_gpu=True)
print(score)

### Effet du nombre d'epochs

Modifiez le nombre d'epochs et observez les performances du réseau.

Que se passe-t-il s'il est trop grand?<br> 
S'il est trop petit?

### Effet de la taille de la batch

Modifiez la taille de la batch et observez l'utilisation de la carte graphique.

Sur quoi est-ce que la taille de la batch semble avoir le plus d'impact?<br>
Est-ce qu'elle impacte les performances?<br>

### Effet du taux d'apprentissage (lr)

Finalement, observez l'impact du taux d'apprentissage sur l'entraînement.

Que se passe-t-il s'il est trop grand?<br> 
S'il est trop petit?

### Défi

Modifiez les hyperparamètres pour améliorer les performances du réseau.<br>
Essayez d'obtenir plus de 65% en test.

Vous pouvez aussi tenter de battre le state of the art: http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html#43494641522d3130